In [6]:
import numpy as np
import tinygrad 
from tinygrad.extra.datasets import fetch_cifar
import time
from tinygrad.lazy import Device
Device.DEFAULT = "CUDA"

import cv2
start =time.monotonic()
X_train, Y_train,= fetch_cifar(train=True)

In [7]:
from tinygrad.nn import Linear, Conv2d
from tinygrad.nn import optim
from tinygrad.tensor import Tensor
Tensor.training= True

class TinyCIFAR:
    def __init__ (self):
        self.l1 = Linear(3072,3072, bias=True)
        self.l2 = Linear(3072,255,bias=True)
        self.l3 = Linear(255,10,bias=True)
    def __call__ (self, x):
        x = self.l1(x)
        x = x.leakyrelu()
        x = self.l2(x)
        x = x.leakyrelu()
        x = self.l3(x)
        return x.log_softmax()
    #here lies the buggers
    #maybe a dense just doesnt learn CIFAR? 
    
net = TinyCIFAR()

In [8]:
print(X_train.shape)
X_trains = X_train.reshape(50000, -1)
print(X_trains.shape)
from tinygrad.nn.optim import SGD, Adam
opt = SGD([net.l1.weight, net.l2.weight, net.l3.weight], lr=3e-2)

def cross_entropy(out, Y):
  num_classes = out.shape[-1]
  YY = Y.flatten().astype(np.int32)
  y = np.zeros((YY.shape[0], num_classes), np.float32)
  y[range(y.shape[0]),YY] = -1.0*num_classes
  y = y.reshape(list(Y.shape)+[num_classes])
  y = Tensor(y)
  return out.mul(y).mean()

# print(Y_train.shape)
# print(X_trains[5,:].max())
# X_trains = np.divide(X_trains, 256)
# print(X_trains[:,:].max())


(50000, 3, 32, 32)
(50000, 3072)


In [9]:




BS=32

#Train the modelieren

for step in range(12000):
    samp=np.random.randint(0, X_trains.shape[0], BS)
    batch = Tensor(X_trains[0], requires_grad=True)
    labels = Y_train[samp]
    out = net(batch)
    loss = cross_entropy(out, labels)
    #print(loss.numpy())
    opt.zero_grad() #NOTE:I'm unsure why it's giving me t.grad is not None (it was array shaping)
    loss.backward()
    opt.step()
    #accuracy dealings
    pred = np.argmax(out.numpy(), axis=-1)
    acc = (pred == labels).mean()
    # if acc >= 1: #how optimistic of myself
    #     break
    if step % 25 == 0:
        print(f'step{step} | loss: {loss.numpy()} | Accuracy: {acc}')
    if step == 2000: 
        if acc < 0.25:
            print('Training but not learning')
            break


end=time.monotonic()
print(f'{end-start:.3}s')



step0 | loss: 94.07295227050781 | Accuracy: 0.15625
step25 | loss: 2.3071348667144775 | Accuracy: 0.0625
step50 | loss: 2.2866902351379395 | Accuracy: 0.1875
step75 | loss: 2.3039605617523193 | Accuracy: 0.0625
step100 | loss: 2.306682825088501 | Accuracy: 0.03125
step125 | loss: 2.290593385696411 | Accuracy: 0.09375
step150 | loss: 2.31390643119812 | Accuracy: 0.0625
step175 | loss: 2.3019955158233643 | Accuracy: 0.0625
step200 | loss: 2.306474447250366 | Accuracy: 0.09375
step225 | loss: 2.3026177883148193 | Accuracy: 0.0625
step250 | loss: 2.2985711097717285 | Accuracy: 0.09375
step275 | loss: 2.3137047290802 | Accuracy: 0.09375
step300 | loss: 2.2967827320098877 | Accuracy: 0.15625
step325 | loss: 2.3128976821899414 | Accuracy: 0.0
step350 | loss: 2.3058290481567383 | Accuracy: 0.0625
step375 | loss: 2.3047659397125244 | Accuracy: 0.0625
step400 | loss: 2.3036835193634033 | Accuracy: 0.09375
step425 | loss: 2.3039236068725586 | Accuracy: 0.125
step450 | loss: 2.3186464309692383 | A

In [10]:
#Validate model
X_test, Y_test= fetch_cifar(train=False)
print(X_test.shape, Y_test.shape)
av_acc = 0 #loopy fucks up without this
X_tests = X_test.reshape(10000, -1)
print(X_tests.shape)
testamount = 100
for step in range(testamount):
    samp = np.random.randint(0, X_tests.shape[0], size=BS)
    batch = Tensor(X_tests[samp], requires_grad=True)
    labels = Y_test[samp]
    out = net (batch)

    pred = np.argmax(out.numpy(), axis=-1)
    av_acc += (pred == labels).mean()
    # print(out.numpy())
    # #print(np.argmax(out.numpy()))
    # print(Y_test[samp])
    # #print(batch.numpy())

print(f"Test Accuracy: {av_acc / testamount}")

(10000, 3, 32, 32) (10000,)
(10000, 3072)
Test Accuracy: 0.094375
